In [1]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt

In [2]:
dev = qml.device("default.qutrit", wires=7)

In [3]:
# State preparation with angle embedding
def state_prep(x):
    for i, angle in enumerate(x):
        qml.TRX(angle, wires=i, subspace=[0, 1])

In [14]:
def rot(*weights, wires):
    # Arbitrary rotation on single wire
    qml.TRY(weights[0], wires=wires, subspace=[0, 1])
    qml.TRY(weights[1], wires=wires, subspace=[0, 2])
    qml.TRY(weights[2], wires=wires, subspace=[0, 1])
    qml.TRZ(weights[3], wires=wires, subspace=[0, 2])
    qml.TRZ(weights[4], wires=wires, subspace=[0, 1])
    qml.TRY(weights[5], wires=wires, subspace=[0, 1])
    qml.TRY(weights[6], wires=wires, subspace=[0, 2])
    qml.TRY(weights[7], wires=wires, subspace=[0, 1])


In [15]:
def layer(weights):
    # Trainable layer
    # Arbitrary rotation on all wires
    qml.broadcast(unitary=rot, pattern="single", wires=list(range(7)), parameters=weights)

    # Entanglement gate
    qml.broadcast(unitary=qml.TAdd, pattern="ring", wires=list(range(7)))

In [16]:
@qml.qnode(dev)
def classifier(weights, x):
    state_prep(x)

    for W in weights:
        layer(W)

    return qml.expval(qml.GellMann(wires=0, index=1))

In [17]:
def cost(weights, X, Y):
    preds = [classifier(weights, x) for x in X]
    loss = 0
    for y, p in zip(Y, preds):
        loss += (y - p) ** 2

    return loss / len(Y)

def accuracy(weights, X, Y):
    preds = [classifier(weights, x) for x in X]
    loss = 0
    for y, p in zip(Y, preds):
        if np.isclose(y, p):
            loss += 1

    return loss / len(Y)

In [18]:
data = np.loadtxt("./seeds_dataset.txt")
X = np.array(data[:, :-1], requires_grad=False)
Y = np.array(data[:, -1], requires_grad=False)
Y = Y - 2

In [19]:
np.random.seed(0)
num_data = len(Y)
num_train = int(0.75 * num_data)
index = np.random.permutation(range(num_data))
X_train = X[index[:num_train]]
Y_train = Y[index[:num_train]]
X_val = X[index[num_train:]]
Y_val = Y[index[num_train:]]

In [20]:
opt = qml.GradientDescentOptimizer(stepsize=0.1)

In [23]:
num_qutrits = 7
num_layers = 1
weights_init = 0.01 * np.random.randn(num_layers, num_qutrits, 8, requires_grad=True)

In [24]:
n_its = 100
weights = weights_init

loss_track = []

for it in range(n_its):
    weights, _loss = opt.step_and_cost(cost, weights, X_train, Y_train)
    if it % 5 == 0:
        print(f"Loss at iteration {it:2} = {_loss: .10f}  Accuracy = {accuracy(weights, X_train, Y_train): .2f}")
    loss_track.append(_loss)

ValueError: Parameters must contain entries for 7 unitaries; got 1 entries